In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
import json
import datetime

plt.rcParams["figure.figsize"] = (20, 5)

In [2]:
data = np.genfromtxt("../로그 데이터/SEG_SGEMM_result.txt", delimiter="\n", dtype=np.int64)
data

array([ 3196231680, 93292771632, 93293300344, ..., 92658792872,
       92658792864, 92654987192], dtype=int64)

In [3]:
delta = np.array([data[i+1] - data[i] for i in range(int(len(data))-1)])
delta, len(delta)

(array([90096539952,      528712,       73032, ...,    36097352,
                 -8,    -3805672], dtype=int64),
 237194)

In [41]:
data_counts = pd.Series(data=delta.T).value_counts()
data_counts

 0             27460
 4096           7851
 909517620      1849
-909517620      1848
 8192           1481
               ...  
-3833072           1
 349797616         1
-5403768464        1
-568288            1
 7770728296        1
Length: 60247, dtype: int64

In [5]:
def category_by_threshold(data_counts, threshold, sign="over"):
    if sign == "under":
        category = (data_counts < threshold)
    else:
        category = (data_counts > threshold)
    return category.index[category == True]

In [15]:
threshold = 50
category_over_threshold = category_by_threshold(data_counts, threshold).to_numpy()
category_over_threshold

array([         0,       4096,  909517620, -909517620,       8192,
               -8,      -4096,          8,      12288,       2416,
            16384,         24,       3520,      -2744,        -12,
           -12288,      20480,         32,         64,          6,
             4104,        -16,      24576,         28,       5280,
            -2884,      28672,         56,      -8192,        240,
             3904,       4092,      32768,       4128, -515913384,
            40960,  515913384,       4080,       -240,     172032,
            -8304,      90112,       3072,         16,      53248,
               -4,         44,          2,        432,      81920,
               48,     118784,       3936,     126976,       4112,
                4,       3856,     -16384,      11776,      61440,
             -432,        256,        -24,        -48,     -32768,
           131072,      69632,     135168,      94208,     -20480,
            -4040,     241664,      -5888,     106496,      98

In [26]:
category_over_threshold.shape

(86,)

In [76]:
minimum = 10
category_over_minimum_total = category_by_threshold(data_counts, minimum).to_numpy()
mask = np.isin(category_over_minimum_total, category_over_threshold)
mask

array([ True,  True,  True, ..., False, False, False])

In [77]:
category_over_minimum = category_over_minimum_total[np.where(mask == False)[0]]
category_over_minimum

array([   159744,      -256,      3872, ..., 117054760, 388317184,
        -5073968], dtype=int64)

In [92]:
closest_category = []
for category in category_over_minimum[:50]:
    closest_category.append(category_over_threshold[np.argmin(np.abs(category_over_threshold - category))])

In [93]:
closest_category[:10], category_over_minimum[:10]

([172032, -240, 3856, 184320, 44, -48, -192512, -8192, 184320, 256],
 array([   159744,      -256,      3872,    200704,        40,       -56,
        -22654976,     -8216,    180224,       320], dtype=int64))

In [94]:
closest_category_index = [np.where(delta == clst_ctg)[0][:10] for clst_ctg in closest_category]
sample_category_index = [np.where(delta == smpl_ctg)[0][:10] for smpl_ctg in sample_category]

In [60]:
def search_slice(category_index, range_value=8):
    result_slice = []
    for ctg_index in category_index:
        tmp = []
        for idx in ctg_index:
            tmp.append(np.r_[delta[idx-range_value:idx], delta[idx+1:idx+range_value+1]])
        result_slice.append(np.array(tmp))
    return result_slice

In [74]:
category_over_threshold_slice = search_slice(category_over_threshold_index)
sample_category_slice = search_slice(sample_category_index)

In [ ]:
from sklearn.metrics import mean_squared_error

result_mse = []
for i in range(len(category_over_threshold_slice)):
    tmp = 0
    for co in category_over_threshold_slice[i]:
        for sc in sample_category_slice[i]:
            tmp =+ mean_squared_error(co, sc)
    result.append(tmp/(category_over_threshold_range[i].shape[0] * category_under_threshold_range[i].shape[0]))

In [17]:
from sklearn.metrics import mean_squared_error

result = []
for i in range(len(category_over_threshold_range)):
    tmp = 0
    for co in category_over_threshold_range[i]:
        for cu in category_under_threshold_range[i]:
            tmp =+ mean_squared_error(co, cu)
    result.append(tmp/(category_over_threshold_range[i].shape[0] * category_under_threshold_range[i].shape[0]))

In [21]:
result[:10]

[308675193173.7431,
 612357242143363.1,
 -11036130921574.953,
 23095138175830.156,
 -155679915491370.25,
 324963002223.6062,
 -89016380415882.12,
 458418993673.54034,
 744343300062.7511,
 455279863322720.0]

In [22]:
np.savetxt("analysis_result.csv", np.array(result))